In [ ]:
using RigidBodySim
using RigidBodyDynamics
using Plots
using LinearAlgebra
using Statistics
using SpecialFunctions
using Swarm
using Trajectory
using PerformanceIndex
using StaticArrays
include("src/fracionario.jl")
include("src/simulation.jl")
include("src/graficos.jl");

In [ ]:
urdf = "src/pelican.urdf"
robot = parse_urdf(urdf) # load dynamic
initial_state = MechanismState(robot) # inicial conditions 
num_joints = num_positions(initial_state); # joints quantity

In [ ]:
mass(robot)

In [ ]:
axis = SVector(0., 1., 0.) # joint axis
I_1 = 0.333 # moment of inertia about joint axis
c_1 = -0.15 # center of mass location with respect to joint axis
m_1 = 5. # mass
frame1 = CartesianFrame3D("new") # the reference frame in which the spatial inertia will be expressed
inertia1 = SpatialInertia(frame1, I_1 * axis * axis', m_1 * SVector(0, 0, c_1), m_1)
body = RigidBody(inertia1)
attach!(robot, last(bodies(robot)), body, Joint("elbow", Fixed{Float64}()) )
remove_fixed_tree_joints!(robot);

In [ ]:
mass(robot)

In [ ]:
time_end = 3.0 # simulation time
Δt = 0.05; # discretization time
saturation_value = [200., 15.]

### GAINS ############

kpz = [ 2500., 100.]
kvz = [180., 10.]
lambdaz = [1., 1.];

kpo = [9202.86, 405.272]
kvo = [161.648, 10.0]
lambdao = [1., 1.]

kpf = [2769.38, 10.0]
kvf = [3016.76, 58.0633]
lambdaf = [0.470696, 0.834637];

In [ ]:
final_position = 0.8
xr, vr, ar, jr = minimumjerk(fill( final_position, num_joints), time_end);

In [ ]:
x, v, a, j, time_out, ta, tj, max_torque = simulationPDDigitalFractional(kpz, kvz, lambdaz, xr, vr, ar, jr, robot,
    initial_state, Δt, time_end, showtorque = true, maxTorque = saturation_value);
ex, ev, ea, ej, time_out, ta, tj = erroPDDigitalFractional(kpz, kvz,lambdaz, xr, vr, ar, jr, robot,
    initial_state, Δt, time_end, maxTorque = saturation_value)

xo, vo, ao, jo, time_outo, tao, tjo, max_torqueo = simulationPDDigitalFractional(kpo, kvo,lambdao, xr, vr, ar,
    jr, robot, initial_state, Δt, time_end, showtorque = true, maxTorque = saturation_value);
exo, evo, eao, ejo, time_outo, tao, tjo = erroPDDigitalFractional(kpo, kvo,lambdao, xr, vr, ar, jr, robot,
    initial_state, Δt, time_end, maxTorque = saturation_value)

xf, vf, af, jf, time_outf, taf, tjf, max_torquef = simulationPDDigitalFractional(kpf, kvf,lambdaf, xr, vr, ar,
    jr, robot, initial_state, Δt, time_end, showtorque = true, maxTorque = saturation_value);
exf, evf, eaf, ejf, time_outf, taf, tjf = erroPDDigitalFractional(kpf, kvf,lambdaf, xr, vr, ar, jr, robot,
    initial_state, Δt, time_end, maxTorque = saturation_value);

In [ ]:
plot_xv(time_out, x, xo, xf,xr, v, vo ,vf, vr, 1)

In [ ]:
plot_aj(ta, tj, a,  ao, af, ar, j, jo,jf, jr, 1)

In [ ]:
plot_xv(time_out, x, xo, xf,xr, v, vo ,vf, vr, 2)

In [ ]:
plot_aj(ta, tj, a,  ao, af, ar, j, jo,jf, jr, 2)